# Folium Heatmap with Optional features 

In [47]:
import numpy as np
import pandas as pd
import datetime
import folium
from folium.plugins import HeatMap

mydf = pd.read_csv("nyc_crash_fol.csv")




#Create a map object
nyc_map = folium.Map(location=[40.751456, -73.979578 ],
                     zoom_start=11,
                     tiles="Cartodbpositron")

#Heat_map
heat_map = folium.FeatureGroup(name = 'heat_map')
max_amount = float(60)
heat_map.add_child( HeatMap (list(zip(mydf['LATITUDE'].values, mydf['LONGITUDE'].values)),
                             min_opacity=0.2,
                             max_val=max_amount,
                             radius=3, blur=2, max_zoom=1))
nyc_map.add_child(heat_map)


def plotter(mydf, my_map, color):
    '''
    Plots geo-locations coordinates(lattitude, longitude) from a DataFrame

    ARGS:
        df -> dataframe containing relevant coordinates
        my_map -> the Feature group the points will be added to
        color -> color of dots
        
    Return:
        none
    '''
    
    for index, row in mydf.iterrows():
        folium.CircleMarker(location=(row['LATITUDE'], row['LONGITUDE']),
                                    radius=4,
                                    color=color,
                                    popup=str('Number of persons killed: ' + str(row['NUMBER OF PERSONS KILLED']) \
                                              + '\nDate: ' + row['CRASH DATE'] \
                                              + '\nTime: '+ row['CRASH TIME'] \
                                              + '\nContrib. Factor: ' + str(row['CONTRIBUTING FACTOR VEHICLE 1'])
                                             ),
                                    fill=True).add_to(my_map)

       
        
#Add a layer for all accidents involving cyclists killed
cyclist_map = folium.FeatureGroup(name = 'cyclist')
cycle_df = mydf[mydf['NUMBER OF CYCLIST KILLED'] > 0]
plotter(cycle_df, cyclist_map, "#e32522") #red dots
nyc_map.add_child(cyclist_map)

#Add a layer for crashes involving pedestrians killed
pedestrian_map = folium.FeatureGroup(name = 'pedestrian_map')
pedestrian_df = mydf[mydf['NUMBER OF PEDESTRIANS KILLED'] > 0]
plotter(pedestrian_df, pedestrian_map, "#00a550") #green dots
nyc_map.add_child(pedestrian_map)

#Add a layer for crashes involving motorists killed
motorist_killed = folium.FeatureGroup(name = 'motorist_killed')
motorist_df = mydf[mydf['NUMBER OF MOTORIST KILLED'] > 0]
plotter(motorist_df, motorist_killed, "#03fc35") #blue dots
nyc_map.add_child(motorist_killed)

#Add a layer for all accidents caused by road rage   
road_rage = folium.FeatureGroup(name = 'road_rage')
road_rage_df = mydf[mydf['CONTRIBUTING FACTOR VEHICLE 1'] == 'Aggressive Driving/Road Rage']
plotter(road_rage_df, road_rage, '#e38f22') #orange dots
nyc_map.add_child(road_rage)

# #Add a layer for all accidents involving a driver distraction  
# distract = folium.FeatureGroup(name = 'distract')
# distract_df = mydf[mydf['CONTRIBUTING FACTOR VEHICLE 1'] == 'Driver Inattention/Distraction']
# plotter(distract_df, distract, '#e38f22') #orange dots
# nyc_map.add_child(distract)


# #Add a layer for all accidents due to following too closely   
# too_close = folium.FeatureGroup(name = 'too_close')
# too_close_df = mydf[mydf['CONTRIBUTING FACTOR VEHICLE 1'] == 'Following Too Closely']
# plotter(too_close_df, too_close, '#1e2100') #yello dots
# nyc_map.add_child(too_close)

#Add toggle buttons for layers
folium.LayerControl().add_to(nyc_map)     #Add layer control to toggle on/off
nyc_map.save('Folium_heat_map.html')
